# Assignment 2: NLP

## Envirnment Setup

Importing the dataset (needs to be done only once per notebook)

In [1]:
# needs to be run only once per notebook
# !wget "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
# !tar -xzf aclImdb_v1.tar.gz

In [2]:
!cat aclImdb/imdb.vocab | wc -l # number of vocab words

89526


## Imports

In [69]:
import nltk # natural language tool kit: for text pre-processing
import os # for listing directories
from bs4 import BeautifulSoup as bs # library for removing html tags from text
import numpy as np # no comment :P
from nltk.corpus import stopwords # a set of common stopwords from nltk
from gensim import models
import gensim
from collections import namedtuple
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import wordnet
from sklearn.model_selection import cross_val_score
from itertools import product

# import helper functions
from helpers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
# download resources for nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [71]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '``', "''", '...'])

## Reading the dataset and preprocessing the text

In [72]:
train_pos = []
train_neg = []
wnl = nltk.WordNetLemmatizer()

In [73]:
train_pos = read_data('aclImdb/train/pos')
print("read {} positive training reviews".format(len(train_pos)))

train_neg = read_data('aclImdb/train/neg')
print("read {} negative training reviews".format(len(train_neg)))

train_data = train_pos + train_neg
print(len(train_data))

read 12500 positive training reviews
read 12500 negative training reviews
25000


In [74]:
test_pos = read_data('aclImdb/test/pos')
print("read {} positive test reviews".format(len(test_pos)))

test_neg = read_data('aclImdb/test/neg')
print("read {} negative test reviews".format(len(test_neg)))

test_data = test_pos + test_neg
print(len(test_data))

read 12500 positive test reviews
read 12500 negative test reviews
25000


In [75]:
train_labels = np.array([1]*12500 + [0]*12500)
test_labels = np.array([1]*12500 + [0]*12500)

## TF-IDF

In [76]:
# put positive and negative training data in one file
!cat aclImdb/train/pos/all.txt aclImdb/train/neg/all.txt > all_train.txt
!cat aclImdb/test/pos/all.txt aclImdb/test/neg/all.txt > all_test.txt

In [77]:
tfidf = TfidfVectorizer()
all_train_file = open('all_train.txt', 'r')
train_doc_matrix = tfidf.fit_transform([review for review in all_train_file.readlines()])

In [78]:
print(len(tfidf.vocabulary_))

67109


In [79]:
all_test_file = open('all_test.txt', 'r')
test_doc_matrix = tfidf.transform([review for review in all_test_file.readlines()])

In [80]:
print(test_doc_matrix.shape)

(25000, 67109)


In [81]:
clf3 = LogisticRegression()
clf3.fit(train_doc_matrix, train_labels)
print(clf3.score(test_doc_matrix, test_labels))

0.87792


In [82]:
word_idfs = dict(zip(tfidf.get_feature_names(), tfidf._tfidf.idf_))

## Word2Vec

In [83]:
word_dimensions = 100
model=Word2Vec(train_data, size = word_dimensions)

In [84]:
train_doc_vecs = get_doc_vecs_for_data(train_data, model.wv, word_dimensions, word_weights=word_idfs)
test_doc_vecs = get_doc_vecs_for_data(test_data, model.wv, word_dimensions, word_weights=word_idfs)

## Classifying using IDF weighted embeddings

In [85]:
clf = LogisticRegression()

clf.fit(train_doc_vecs, train_labels)
clf.score(test_doc_vecs, test_labels)

0.80712

## Using pretrained GloVe Embeddings

In [86]:
glove_dict_300 = load_glove_dict(300)

In [ ]:
glove_train_doc_vecs = get_doc_vecs_for_data(train_data, glove_dict_300, 300)
glove_test_doc_vecs = get_doc_vecs_for_data(test_data, glove_dict_300, 300)

In [ ]:
%reset_selective -f glove_dict_300

In [ ]:
clf2 = LogisticRegression()
clf2.fit(glove_train_doc_vecs, train_labels)
clf2.score(glove_test_doc_vecs, test_labels)

## Hyper parameter tuning and cross fold validation

In [ ]:
logistic_params = {'C': [0.1, 0.5, 1, 5, 10]}
random_forest_params = {'n_estimators': [10,50,100,300]}

lr_clfs = get_clfs_for_combinations(LogisticRegression, logistic_params)
rf_clfs = get_clfs_for_combinations(RandomForestClassifier, random_forest_params)

# TF-IDF
scores_dict = cross_validate(train_doc_matrix, train_labels, lr_clfs)

In [ ]:
for clf in scores_dict.keys():
    params = []
    for param in logistic_params.keys():
        params.append( (param, clf.get_params()[param]) )
    print("clf with params: {}, score: {}".format(params, scores_dict[clf]))
    